In [24]:
import intake,yaml
import intake_esm
import numpy as np
#anonymous True access to test data s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1/v20190726

In [2]:
storage_options = {'anon': True}

In [5]:
col_url = "gfdltest.json" 

In [6]:
col = intake.open_esm_datastore(col_url)


In [7]:
col.df

,product_id,institute,model,experiment,frequency,modeling_realm,mip_table,ensemble_member,variable,temporal_subset,version,path
0,output,NOAA-GFDL,GFDL-ESM4,historical,mon,atmos,Amon,r1i1p1,tas,185001-194912,v20190726,s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...
1,output,NOAA-GFDL,GFDL-ESM4,historical,mon,atmos,Amon,r1i1p1,tas,195001-201412,v20190726,s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...


In [8]:
expname_filter = ['historical']
table_id_filter = 'Amon'
model_filter = 'GFDL-ESM4'
variable_id_filter = "tas"

In [9]:
cat = col.search(experiment=expname_filter, mip_table=table_id_filter,model=model_filter)


In [10]:
type(cat.df.head(1))
type(cat)
cat.df.head(1)


,product_id,institute,model,experiment,frequency,modeling_realm,mip_table,ensemble_member,variable,temporal_subset,version,path
0,output,NOAA-GFDL,GFDL-ESM4,historical,mon,atmos,Amon,r1i1p1,tas,185001-194912,v20190726,s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...


In [15]:
dset_dict = col.to_dataset_dict(cdf_kwargs={'chunks': {'time': 20}}, storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'product_id.institute.model.modeling_realm.experiment.frequency.mip_table'


In [27]:
hxr = dset_dict["output.NOAA-GFDL.GFDL-ESM4.atmos.historical.mon.Amon"]

In [29]:
lat1 = 60
lat2 = 90
tas = hxr.tas
weights = np.cos(np.deg2rad(tas.lat))
weights.name = "weights"

In [30]:
hgtas = hxr.tas.sel(time=slice("1994", "2014"))
hgtas = hgtas.weighted(weights)
hgtas = hgtas.mean(dim=['lon','lat','time'])
hgtas = hgtas.values
print(hgtas) #(287.2) from ferret

[287.16173045]


In [ ]:
hatas = hxr.tas.sel(time=slice("1994", "2014"),lat=slice(lat1, lat2))
hatas = hatas.weighted(weights)
hatas = hatas.mean(dim=['lon','lat','time'])
hatas = hatas.values 
print(hatas)#hatas 264.5 from Ferret